# Advanced pandas - Going Beyond the Basics

## polars
___

### Table of Contents
1. [Import dependencies](#section1)
2. [Read data into polars](#section2)
3. [Expressions and operations](#section3)  
    3.1 [select()](#section3.1)  
    3.2 [with_columns()](#section3.2)  
    3.3 [filter()](#section3.3)  
    3.4 [groupby()](#section3.4)  
    3.5 [Chaining expressions](#section3.5)  
4. [Joining and Concatenating DataFrames](#section4)

___
<a id='section1'></a>
# (1) Import dependencies

In [1]:
# Install dependencies (if not already done so)
# !pip install pandas==2.0.3
# !pip install polars==0.18.7

In [1]:
import numpy as np
import pandas as pd
import polars as pl
from datetime import datetime

___
<a id='section2'></a>
# (2) Read data into `polars`
- Data Source: https://archive.ics.uci.edu/dataset/352/online+retail ((CC BY 4.0) license)

`polars` supports reading and writing to all common file formats (e.g. CSV, JSON, Parquet), cloud storage (S3, Azure Blob, BigQuery) and databases (e.g. PostgreSQL, MySQL etc.).

In this example, we will use the fast CSV reading function of `polars`, as shown below:

In [32]:
# Read CSV in polars
df = pl.read_csv('https://raw.githubusercontent.com/kennethleungty/Educative-Advanced-Pandas/main/data/csv/online_retail_dataset.csv',
                 encoding='utf-8', # Ensure values are encoded appropriately
                 ignore_errors=True # Hide errors first since dtypes not specified
                )

# View output
df.head()

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
i64,str,str,i64,str,f64,i64,str
536365,"""85123A""","""WHITE HANGING …",6,"""1/12/2010 8:26…",2.55,17850,"""United Kingdom…"
536365,"""71053""","""WHITE METAL LA…",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"
536365,"""84406B""","""CREAM CUPID HE…",8,"""1/12/2010 8:26…",2.75,17850,"""United Kingdom…"
536365,"""84029G""","""KNITTED UNION …",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"
536365,"""84029E""","""RED WOOLLY HOT…",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"


`polars` has a strict schema, meaning that data types should be known before running the query. In the above case, because we did not specify the `dtypes`, the data type inference was done automatically and the output DataFrame indicates the inferred `dtype` for each column.

To specify the data types appropriately for `polars` to work optimally, we can use the `dtypes` and `columns` parameters, as illustrated below:

In [3]:
# Specify column names
columns = ["InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", 
           "UnitPrice", "CustomerID", "Country"]

# Define list of data types in the same sequence as the columns
dtypes_list = [pl.Utf8, pl.Utf8, pl.Utf8, pl.Int64, pl.Datetime,
               pl.Float64, pl.Int64, pl.Categorical]

# Read CSV in polars with dtypes specified
df = pl.read_csv('https://raw.githubusercontent.com/kennethleungty/Educative-Advanced-Pandas/main/data/csv/online_retail_dataset.csv',
                 encoding='utf-8', # Ensure values are encoded appropriately
                 dtypes=dtypes_list,
                 columns=columns
                 )

# View output
df.head()

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
str,str,str,i64,datetime[μs],f64,i64,cat
"""536365""","""85123A""","""WHITE HANGING …",6,null,2.55,17850,"""United Kingdom…"
"""536365""","""71053""","""WHITE METAL LA…",6,null,3.39,17850,"""United Kingdom…"
"""536365""","""84406B""","""CREAM CUPID HE…",8,null,2.75,17850,"""United Kingdom…"
"""536365""","""84029G""","""KNITTED UNION …",6,null,3.39,17850,"""United Kingdom…"
"""536365""","""84029E""","""RED WOOLLY HOT…",6,null,3.39,17850,"""United Kingdom…"


> **Note**: `pl.Utf8` represents the UTF-8 encoded string type in `polars`.The complete list of `polars` data types can be found in the section below on the Educative lesson page.

If we do not wish to read the entire large dataset directly, we can do scanning with `scan_csv()` instead. `scan_csv()` delays the parsing of the dataset, and instead lazily reads it and returns a holder known as a `LazyFrame` (rather than a DataFrame). A `LazyFrame` is a representation of a lazy computation graph/query against a DataFrame. 

The purpose of this is to let `polars` generate an optimal execution plan before actually executing the transformation, so that `polars` can skip over certain columns if they are not needed in the computation. The actual computation takes place when the `collect()` is called, as shown below:

In [26]:
lazy_df = pl.scan_csv('data/online_retail_dataset.csv', # Lazy read must be from local file, not external URL
                      encoding='utf8',
                      ignore_errors = True)
type(lazy_df)

polars.lazyframe.frame.LazyFrame

In [27]:
# Retrieve data when run collect()
df = lazy_df.collect(streaming=True)

# View output
df.head()

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
i64,str,str,i64,str,f64,i64,str
536365,"""85123A""","""WHITE HANGING …",6,"""1/12/2010 8:26…",2.55,17850,"""United Kingdom…"
536365,"""71053""","""WHITE METAL LA…",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"
536365,"""84406B""","""CREAM CUPID HE…",8,"""1/12/2010 8:26…",2.75,17850,"""United Kingdom…"
536365,"""84029G""","""KNITTED UNION …",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"
536365,"""84029E""","""RED WOOLLY HOT…",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"


Lazy loading with `scan_csv()` is as opposed to the eager loading with `read_csv()` we use in `pandas`.

> **Note**: An additional benefit of lazy evaluation is that it allows queries to be executed in a streaming manner. Instead of processing the data all-at-once, `polars` can execute the query in batches so that we can process datasets that are larger-than-memory. To tell `polars` we want to execute a query in streaming mode, we pass the `streaming=True` argument as shown above.

> **Note**: One caveat about streaming is that it does not work for all operations (although it does for core operations like filter, groupby and join). If streaming is not available for some operations, `polars` will default to non-streaming and we may run out of memory with a large dataset.

For debugging purposes, sometimes it is useful to just return a few rows to examine the output. We can use the `fetch()` method to return the first n rows, as shown below:

In [6]:
lazy_df.fetch(3)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
i64,str,str,i64,str,f64,i64,str
536365,"""85123A""","""WHITE HANGING …",6,"""1/12/2010 8:26…",2.55,17850,"""United Kingdom…"
536365,"""71053""","""WHITE METAL LA…",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"
536365,"""84406B""","""CREAM CUPID HE…",8,"""1/12/2010 8:26…",2.75,17850,"""United Kingdom…"


> **Note**: To showcase the lazy evaluation capabilities of `polars`, we will load the dataset locally for the subsequent examples, instead of retrieving it from the GitHub repo link.

___
<a id='section3'></a>
# (3) Expressions and operations

`polars` has a powerful concept called **expressions**, and it is central to its fast performance. Expressions are at the core of many data science operations as they are used to represent operations performed on one or more columns in a DataFrame. 

They can include basic arithmetic, aggregations, comparisons, and other more complex transformations. When using `polars`, expressions allow users to create concise, readable code for data manipulation. `polars` will also perform these core data transformations very quickly by automatic query optimization on each expression and automatic parallelization of expressions on many columns.

In this lesson, let us focus on four of these expression methods, namely:
1. `select()`
2. `with_columns()`
3. `filter()`
4. `groupby()`
___

<a id='section3.1'></a>
## (3.1) `select()`
We can select all or specific columns using the `pl.col()` expression shown below:

In [7]:
# Select all columns (asterisk * refers to all columns) (Option 1)
df.select(pl.col('*'))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
i64,str,str,i64,str,f64,i64,str
536365,"""85123A""","""WHITE HANGING …",6,"""1/12/2010 8:26…",2.55,17850,"""United Kingdom…"
536365,"""71053""","""WHITE METAL LA…",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"
536365,"""84406B""","""CREAM CUPID HE…",8,"""1/12/2010 8:26…",2.75,17850,"""United Kingdom…"
536365,"""84029G""","""KNITTED UNION …",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"
536365,"""84029E""","""RED WOOLLY HOT…",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"
536365,"""22752""","""SET 7 BABUSHKA…",2,"""1/12/2010 8:26…",7.65,17850,"""United Kingdom…"
536365,"""21730""","""GLASS STAR FRO…",6,"""1/12/2010 8:26…",4.25,17850,"""United Kingdom…"
536366,"""22633""","""HAND WARMER UN…",6,"""1/12/2010 8:28…",1.85,17850,"""United Kingdom…"
536366,"""22632""","""HAND WARMER RE…",6,"""1/12/2010 8:28…",1.85,17850,"""United Kingdom…"


In [54]:
# Select all columns (Option 1)
df.select(pl.all())

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
i64,str,str,i64,str,f64,i64,str
536365,"""85123A""","""WHITE HANGING …",6,"""1/12/2010 8:26…",2.55,17850,"""United Kingdom…"
536365,"""71053""","""WHITE METAL LA…",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"
536365,"""84406B""","""CREAM CUPID HE…",8,"""1/12/2010 8:26…",2.75,17850,"""United Kingdom…"
536365,"""84029G""","""KNITTED UNION …",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"
536365,"""84029E""","""RED WOOLLY HOT…",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…"
536365,"""22752""","""SET 7 BABUSHKA…",2,"""1/12/2010 8:26…",7.65,17850,"""United Kingdom…"
536365,"""21730""","""GLASS STAR FRO…",6,"""1/12/2010 8:26…",4.25,17850,"""United Kingdom…"
536366,"""22633""","""HAND WARMER UN…",6,"""1/12/2010 8:28…",1.85,17850,"""United Kingdom…"
536366,"""22632""","""HAND WARMER RE…",6,"""1/12/2010 8:28…",1.85,17850,"""United Kingdom…"


In [8]:
# Select specific columns (Option 1)
df.select(pl.col(['InvoiceNo', 'InvoiceDate']))

InvoiceNo,InvoiceDate
i64,str
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536366,"""1/12/2010 8:28…"
536366,"""1/12/2010 8:28…"


In [9]:
# Select specific columns (Option 2). Useful if we want to apply specific transformations on each column separately
df.select([pl.col('InvoiceNo'), pl.col('InvoiceDate')])

InvoiceNo,InvoiceDate
i64,str
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536366,"""1/12/2010 8:28…"
536366,"""1/12/2010 8:28…"


We can easily combine the selection with other methods, such as `sort()` to sort the values and `limit()` to limit the number of rows returned, as illustrated below:

In [57]:
df.select(
          [pl.col('InvoiceNo'), 
           pl.col('InvoiceDate')]).sort('InvoiceNo', descending=True).limit(5)

InvoiceNo,InvoiceDate
i64,str
544795,"""23/2/2011 13:2…"
544795,"""23/2/2011 13:2…"
544795,"""23/2/2011 13:2…"
544795,"""23/2/2011 13:2…"
544795,"""23/2/2011 13:2…"


If we want to exclude specific columns instead of specifying the ones we want to include, as we leverage `pl.exclude()` as shown below:

In [11]:
df.select([pl.exclude('Description', 'Country', 'InvoiceDate')]).limit(5)

InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID
i64,str,i64,f64,i64
536365,"""85123A""",6,2.55,17850
536365,"""71053""",6,3.39,17850
536365,"""84406B""",8,2.75,17850
536365,"""84029G""",6,3.39,17850
536365,"""84029E""",6,3.39,17850


We can even select by data types:

In [12]:
# Obtain columns that are int64 and string
df.select(pl.col(pl.Int64, pl.Utf8)).limit(5)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country
i64,str,str,i64,str,i64,str
536365,"""85123A""","""WHITE HANGING …",6,"""1/12/2010 8:26…",17850,"""United Kingdom…"
536365,"""71053""","""WHITE METAL LA…",6,"""1/12/2010 8:26…",17850,"""United Kingdom…"
536365,"""84406B""","""CREAM CUPID HE…",8,"""1/12/2010 8:26…",17850,"""United Kingdom…"
536365,"""84029G""","""KNITTED UNION …",6,"""1/12/2010 8:26…",17850,"""United Kingdom…"
536365,"""84029E""","""RED WOOLLY HOT…",6,"""1/12/2010 8:26…",17850,"""United Kingdom…"


As an additional demonstration of lazy loading, the following example shows how we can first scan and lazy load our dataset before running `collect()` to execute our `select()` expression:

In [31]:
# Lazy read (scan_csv)
lazy_df = pl.scan_csv('data/online_retail_dataset.csv', # Must be read from local file, not external URL
                      encoding='utf8',
                      ignore_errors = True)

# Select the inclusion of specific columns
df_2 = lazy_df.select([pl.col('InvoiceNo'), pl.col('InvoiceDate')])

# Execute selection with collect()
df_2.collect().head()

InvoiceNo,InvoiceDate
i64,str
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"


> **Note:** This lazy loading before execution with `collect()` applies to all the expressions mentioned in this lesson, beyond just the `select()` example shown here.

___
<a id='section3.2'></a>
## (3.2) `with_columns()`
We can create new columns using `with_columns()`, with the examples shown below where we apply basic operations together with the expressions. The `alias()` method is used to assign the new column an alias name.

In [13]:
df.with_columns([
                 (pl.col('Quantity') + 10).alias('Quantity + 10'), \
                 (pl.col('Quantity').mean()).alias('Average Quantity')
                ]).limit(5)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Quantity + 10,Average Quantity
i64,str,str,i64,str,f64,i64,str,i64,f64
536365,"""85123A""","""WHITE HANGING …",6,"""1/12/2010 8:26…",2.55,17850,"""United Kingdom…",16,12.206492
536365,"""71053""","""WHITE METAL LA…",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…",16,12.206492
536365,"""84406B""","""CREAM CUPID HE…",8,"""1/12/2010 8:26…",2.75,17850,"""United Kingdom…",18,12.206492
536365,"""84029G""","""KNITTED UNION …",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…",16,12.206492
536365,"""84029E""","""RED WOOLLY HOT…",6,"""1/12/2010 8:26…",3.39,17850,"""United Kingdom…",16,12.206492


We also can use `with_columns()` to change the data type of a column. For example, we can convert the "InvoiceDate" column into the appropriate datetime format, as shown below:

In [14]:
# Convert to InvoiceDate column from string to datetime format
df = df.with_columns(pl.col('InvoiceDate').str.to_datetime(format="%d/%m/%Y %H:%M"))

# View first 5 rows
df.head()

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
i64,str,str,i64,datetime[μs],f64,i64,str
536365,"""85123A""","""WHITE HANGING …",6,2010-12-01 08:26:00,2.55,17850,"""United Kingdom…"
536365,"""71053""","""WHITE METAL LA…",6,2010-12-01 08:26:00,3.39,17850,"""United Kingdom…"
536365,"""84406B""","""CREAM CUPID HE…",8,2010-12-01 08:26:00,2.75,17850,"""United Kingdom…"
536365,"""84029G""","""KNITTED UNION …",6,2010-12-01 08:26:00,3.39,17850,"""United Kingdom…"
536365,"""84029E""","""RED WOOLLY HOT…",6,2010-12-01 08:26:00,3.39,17850,"""United Kingdom…"


___
<a id='section3.3'></a>
## (3.3) `filter()`

We can apply conditional filtering on our DataFrames with the `filter()` method. For instance, we can filter our DataFrame to rows where the quantity is greater than 10, invoice number is not null, and unit price is less than or equal to $3.15, as shown below:

In [15]:
df.filter(
          (pl.col('Quantity') > 10) & \
          (pl.col('InvoiceNo').is_not_nan()) & \
          (pl.col('UnitPrice') <= 3.15) 
          ).limit(5)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
i64,str,str,i64,datetime[μs],f64,i64,str
536367,"""84879""","""ASSORTED COLOU…",32,2010-12-01 08:34:00,1.69,13047,"""United Kingdom…"
536370,"""21724""","""PANDA AND BUNN…",12,2010-12-01 08:45:00,0.85,12583,"""France"""
536370,"""21883""","""STARS GIFT TAP…",24,2010-12-01 08:45:00,0.65,12583,"""France"""
536370,"""10002""","""INFLATABLE POL…",48,2010-12-01 08:45:00,0.85,12583,"""France"""
536370,"""21791""","""VINTAGE HEADS …",24,2010-12-01 08:45:00,1.25,12583,"""France"""


Here is another example where we filter by a date range:

In [18]:
df.filter(pl.col('InvoiceDate').is_between(datetime(2010, 12, 5), datetime(2010, 12, 6))).limit(10)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
i64,str,str,i64,datetime[μs],f64,i64,str
537037,"""22114""","""HOT WATER BOTT…",8,2010-12-05 10:03:00,3.95,17243,"""United Kingdom…"
537037,"""21485""","""RETROSPOT HEAR…",6,2010-12-05 10:03:00,4.95,17243,"""United Kingdom…"
537037,"""22835""","""HOT WATER BOTT…",3,2010-12-05 10:03:00,4.65,17243,"""United Kingdom…"
537037,"""84029E""","""RED WOOLLY HOT…",6,2010-12-05 10:03:00,3.75,17243,"""United Kingdom…"
537037,"""84029G""","""KNITTED UNION …",6,2010-12-05 10:03:00,3.75,17243,"""United Kingdom…"
537037,"""22158""","""3 HEARTS HANGI…",8,2010-12-05 10:03:00,2.95,17243,"""United Kingdom…"
537037,"""22633""","""HAND WARMER UN…",24,2010-12-05 10:03:00,2.1,17243,"""United Kingdom…"
537037,"""22988""","""SOLDIERS EGG C…",12,2010-12-05 10:03:00,1.25,17243,"""United Kingdom…"
537037,"""82494L""","""WOODEN FRAME A…",6,2010-12-05 10:03:00,2.95,17243,"""United Kingdom…"


> **Note**: There are many other expression methods in `polars` (e.g., `is_between()`, `is_not_nan()`) that are available and outside the scope of this lesson. Please refer to the `polars` documentation to discover the full list of methods available.

One more example on filtering based on string values:

In [40]:
# Find all transactions with item descriptions that contain the word 'EDWARDIAN'
df.filter(pl.col('Description').str.contains(r'EDWARDIAN'))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
i64,str,str,i64,str,f64,i64,str
536373,"""20679""","""EDWARDIAN PARA…",6,"""1/12/2010 9:02…",4.95,17850,"""United Kingdom…"
536375,"""20679""","""EDWARDIAN PARA…",6,"""1/12/2010 9:32…",4.95,17850,"""United Kingdom…"
536381,"""22774""","""RED DRAWER KNO…",24,"""1/12/2010 9:41…",1.25,15311,"""United Kingdom…"
536381,"""22771""","""CLEAR DRAWER K…",24,"""1/12/2010 9:41…",1.25,15311,"""United Kingdom…"
536381,"""15056BL""","""EDWARDIAN PARA…",2,"""1/12/2010 9:41…",5.95,15311,"""United Kingdom…"
536381,"""15056N""","""EDWARDIAN PARA…",2,"""1/12/2010 9:41…",5.95,15311,"""United Kingdom…"
536396,"""15056BL""","""EDWARDIAN PARA…",6,"""1/12/2010 10:5…",4.95,17850,"""United Kingdom…"
536396,"""20679""","""EDWARDIAN PARA…",6,"""1/12/2010 10:5…",4.95,17850,"""United Kingdom…"
536404,"""22771""","""CLEAR DRAWER K…",12,"""1/12/2010 11:2…",1.25,16218,"""United Kingdom…"


___
<a id='section3.4'></a>
## (3.4) `groupby()`
We can also easily perform the useful groupby operations coupled with aggregation functions, as shown in the examples below:

In [30]:
# Groupby each customer and count how many transactions each customer had
df.groupby('CustomerID', maintain_order=True).count().limit(10)

CustomerID,count
i64,u32
17850,312
13047,44
12583,52
13748,1
15100,6
15291,35
14688,85
17809,17
15311,718


In [37]:
# Groupby customer ID and apply multiple aggregations with agg()
df.groupby('CustomerID', maintain_order=True).agg([
                pl.col('CustomerID').count().alias("Transaction Count"),
                pl.col('UnitPrice').mean().alias("Average Unit Price"),
                pl.col('Quantity').sum().alias("Total Item Quantity"),
                ]).limit(10)

CustomerID,Transaction Count,Average Unit Price,Total Item Quantity
i64,u32,f64,i64
17850,312,3.924712,1693
13047,44,4.251136,311
12583,52,1.869038,1386
13748,1,2.55,80
15100,6,10.95,58
15291,35,6.209143,325
14688,85,1.758706,1245
17809,17,4.044118,442
15311,718,2.689694,7713


___
<a id='section3.5'></a>
## (3.5) Chaining expressions

Finally, let us look at how we can combine expression. The power of expressions is that every expression produces a new expression, which means that they can be chained together in a form of pipeline. For example, we earlier saw how we can lazy load a dataset, apply a `select()` expression, before executing it with `collect()`. The following code shows how we can chain them together into a single expression:

In [43]:
pl.scan_csv('data/online_retail_dataset.csv', 
            encoding='utf8',
            ignore_errors = True) \
  .select([pl.col('InvoiceNo'), pl.col('InvoiceDate')]) \
  .collect(streaming=True) \
  .limit(5)

InvoiceNo,InvoiceDate
i64,str
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"
536365,"""1/12/2010 8:26…"


> **Note**: We generally want to stay in lazy mode for as long as possible (ideally for our entire query) so that `polars` can apply query optimisation. It means that we should aim to place all our necessary data manipulation expressions and queries right up to the `collect()` method.

We can also run a series of separate expressions for each column, as shown below where we have a column where we sort the unique "CustomerID" values and another column where we display the average unit price of items with the word 'RED' in their description:

In [50]:
df.select(pl.col('CustomerID').sort().unique().head(5), 
          pl.col('UnitPrice').filter(pl.col('Description').str.contains(r'RED')).mean()
         )

CustomerID,UnitPrice
i64,f64
12346,4.06338
12347,4.06338
12348,4.06338
12350,4.06338
12352,4.06338


> **Note**: All the expressions above are run in parallel, meaning that separate `polars` expressions are parallelized

Here is another example of chaining expressions, where we generate a new column for total price (calculated from quantity multiplied by unit price) and then select columns that exclude the "Description" and "InvoiceDate" columns:

In [53]:
df.with_columns([(pl.col('Quantity') * pl.col('UnitPrice')).alias('Total Price')])\
                .select(
                        [pl.all().exclude(['Description', 'InvoiceDate'])]
                        )\
                .limit(10)

InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Total Price
i64,str,i64,f64,i64,str,f64
536365,"""85123A""",6,2.55,17850,"""United Kingdom…",15.3
536365,"""71053""",6,3.39,17850,"""United Kingdom…",20.34
536365,"""84406B""",8,2.75,17850,"""United Kingdom…",22.0
536365,"""84029G""",6,3.39,17850,"""United Kingdom…",20.34
536365,"""84029E""",6,3.39,17850,"""United Kingdom…",20.34
536365,"""22752""",2,7.65,17850,"""United Kingdom…",15.3
536365,"""21730""",6,4.25,17850,"""United Kingdom…",25.5
536366,"""22633""",6,1.85,17850,"""United Kingdom…",11.1
536366,"""22632""",6,1.85,17850,"""United Kingdom…",11.1


___
<a id='section4'></a>
# (4) Joining and concatenating DataFrames

Let us now take a look at how we can combine DataFrames in `polars`, starting with the join operation. `polars` supports all types of join (e.g. left, right, inner, outer). The table below shows the full set of join strategies we can adopt:

| Strategy | Description                                                                                                                                           |
|----------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| inner    | Returns row with matching keys in both frames.                                        |
| left     | Returns all rows in the left dataframe, whether or not a match in the right-frame is found.   |
| outer    | Returns all rows from both left and right DataFrames. If no match is found in one frame, columns from the other frame are null-filled.               |
| cross    | Returns Cartesian product of all rows from the left frame with all rows from the right frame. Duplicates rows are retained. |
| asof     | A left-join in which the match is performed on the nearest key rather than on equal keys.                                                            |
| semi     | Returns all rows from the left frame in which the join key is also present in the right frame.                                                        |
| anti     | Returns all rows from the left frame in which the join key is not present in the right frame.                                                         |


For example, the code below shows how we can perform a left join of two `polars` DataFrames:

In [61]:
# Generate two mock polars DataFrames
df_customers = pl.DataFrame(
    {
        "customer_id": [100, 101, 102],
        "name": ["Jane", "Pauline", "Anderson"],
    })

df_orders = pl.DataFrame(
    {   "customer_id": [100, 101, 102],
        "order_id": ["a", "b", "c"],
        "amount_paid": [54.75, 22.3, 89.3],
    })

In [62]:
# Perform left join (syntax similar to pandas)
df_left = df_customers.join(df_orders, 
                          on="customer_id", 
                          how="left")

# View output
df_left

customer_id,name,order_id,amount_paid
i64,str,str,f64
100,"""Jane""","""a""",54.75
101,"""Pauline""","""b""",22.3
102,"""Anderson""","""c""",89.3


We can also concatenate two `polars` DataFrames with the `concat()` method. Vertical concatenation will make the DataFrame longer, whereas horizontal concatenation will make the DataFrame wider. For instance, the example below shows how to perform horizontal concatenation of our two mock DataFrames:

In [66]:
pl.concat([df_customers, 
           df_orders.drop('customer_id')], # Drop customer_id as concat does not allow duplicate columns
           how='horizontal')

customer_id,name,order_id,amount_paid
i64,str,str,f64
100,"""Jane""","""a""",54.75
101,"""Pauline""","""b""",22.3
102,"""Anderson""","""c""",89.3


The other arguments we can feed into the `how` parameter for the concatenation strategy include `vertical`, `align`, and `diagonal`.

> **Note**: There are many other transformation operations that are possible in `polars`, such as melt and pivot.

___
# Wrapping it up
We have only explored the tip of the iceberg in the powerful `polars` package. To gain an even deeper understanding of `polars`, it is highly recommended to visit their comprehensive documentation page [here](https://pola-rs.github.io/polars-book/user-guide/).